A股（至少是指数）是否有周内效应呢？就是特定周几盈利高一些，让我们来验证一下吧。

In [1]:
import pandas as pd
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
from datetime import datetime
import json
import numpy as np
# import cnfundutils
# import libtrdb2
# import trading2_pb2
# import tradingdb2_pb2
# import tradingdb2_pb2_grpc
import libtrading2

import plotly.express as px

isStaticImg = True

cnfundutils.init2()

trdb2cfg = libtrading2.config.loadConfig('./trdb2.yaml')

ModuleNotFoundError: No module named 'tradingdb2_pb2'

In [ ]:
paramsaip = libtrading2.trading2_pb2.AIPParams(
    money=10000,
    type=trading2_pb2.AIPTT_MONTHDAY,
    day=1,
)    

paramsinit = libtrading2.trading2_pb2.InitParams(
    money=10000,
)

buyup = libtrading2.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ema.30'],
)

buydown = libtrading2.trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['down'],
    strVals=['ema.30'],
)

lststart = [1, 2, 3, 4, 5]
lstend = [2, 3, 4, 5, 1]
lsttitle = ['周一', '周二', '周三', '周四', '周五']
lstparams = []
assets = ['cnfunds.510310']
for day in range(0, 4):
    for i in range(0, 5):
        buy0 = libtrading2.trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[lststart[i]],
        )

        sell0 = libtrading2.trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[libtrdb2.nextWeekDay(lststart[i], day + 1)],
        )

        paramsbuy = libtrading2.trading2_pb2.BuyParams(
            perHandMoney=1,
        )

        paramssell = libtrading2.trading2_pb2.SellParams(
            perVolume=1,
        )
        
        s0 = libtrading2.trading2_pb2.Strategy(
            name="normal",
            asset=libtrdb2.str2Asset(assets[0]),
        )

        s0.buy.extend([buy0, buyup])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(libtrading2.trading2_pb2.SimTradingParams(
            assets=[libtrdb2.str2Asset('cnfunds.510310')],
            startTs=0,
            endTs=-1,
            strategies=[s0],
            title='{}{}up'.format(day, i),
        ))
        
        s1 = libtrading2.trading2_pb2.Strategy(
            name="normal",
            asset=libtrdb2.str2Asset(assets[0]),         
        )

        s1.buy.extend([buy0, buydown])
        s1.sell.extend([sell0])
        s1.paramsBuy.CopyFrom(paramsbuy)
        s1.paramsSell.CopyFrom(paramssell)
        s1.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(libtrading2.trading2_pb2.SimTradingParams(
            assets=[libtrdb2.str2Asset('cnfunds.510310')],
            startTs=0,
            endTs=-1,
            strategies=[s1],
            title='{}{}down'.format(day, i),
        ))        
        
lstpnlaip = libtrading2.trdb2.simTradings(trdb2cfg, lstparams)

# # baseline    
# buy0 = trading2_pb2.CtrlCondition(
#     name='buyandhold',
# )

# paramsbuy = trading2_pb2.BuyParams(
#     perHandMoney=1,
# )

# paramsinit = trading2_pb2.InitParams(
#     money=10000,
# )

# pnl = libtrdb2.simTradingEx3(trdb2cfg, assets, 0, -1, [buy0], None, paramsbuy, None, paramsinit, None)
# lstpnlaip.append({'title': 'baseline', 'pnl': pnl})

dfaip = libtrdb2.buildPNLReport(lstpnlaip)

In [ ]:
dfaip[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)